In [10]:
import requests
import ipaddress
import pandas as pd
import numpy as np 
from concurrent.futures import ThreadPoolExecutor, as_completed
from anticaptchaofficial.recaptchav2proxyless import *
import time
import os 

url_fichas = "https://cnslp.gob.bo:3333/api/afiliaciones/afiliado/registro" 
url_oficinas = "https://cnslp.gob.bo:3333/api/afiliaciones/oficinas"
url_disponibilidad = "https://cnslp.gob.bo:3333/api/afiliaciones/atencion/disponible/1"
download_directory = "/media/F/sevima/SEVIMAworkflow-master/fichasCNS/FICHAS"

oficinas = requests.get(url_oficinas,  verify=False )
oficinasDF = pd.DataFrame(oficinas.json()["respuesta"]["data"])



# Generando los elementos 
from selenium import webdriver

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", download_directory)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip,application/pdf,document/pdf")
profile.set_preference( "pdfjs.disabled", True );

navegador = webdriver.Firefox(profile)

time.sleep(5)

qr = "A5l2lu|CNS|18|02/02/2021|11:36|Modulo 2|SUPLICIO SERRANO QUISPE|Regional La Paz"
url_i = "http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha={nro_ficha}&fecha_atencion={fecha_atencion}&hora_aproximada={hora_aproximada}&cubiculo={cubiculo}&persona={persona}&oficina={oficina}&codigo={codigo}".format(
            nro_ficha     = qr.split("|")[2]
            ,fecha_atencion   = qr.split("|")[3]
            ,hora_aproximada    = qr.split("|")[4]
            ,cubiculo  = qr.split("|")[5]
            ,persona  = qr.split("|")[6]
            ,oficina = qr.split("|")[7] 
            ,codigo = qr.split("|")[0])
navegador.get(url_i)
time.sleep(5)



def get_cookies(pc_i):
    data_ip = {}
    data_ip["ip"] = {"ip" : pc_i["ip"]}
    disponible = requests.put(url_disponibilidad+ "/normal",data=json.dumps(data_ip), headers = pc_i["head"],  verify=False )
    j = np.random.randint(1,5)
    time.sleep(j)
    return pc_i, disponible.json()

def ResolveCaptcha():
    start = time.time()
    solver = recaptchaV2Proxyless()
    solver.set_key("1abe93053bf95bd161451cde01bf8e02")
    solver.set_website_url("https://cnslp.gob.bo/")
    solver.set_website_key("6LdXcaYZAAAAADVL-ITTxLsKBGpVoBO_pXoZ468Z")
    # solver.set_verbose(1)
    g_response = solver.solve_and_return_solution()
    print(f'Time taken captcha_i: {time.time() - start}')
    return g_response

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


# Construyendo PCs random 

In [11]:
# Cargando Elementos 
RANGOS_LP = pd.read_csv("ISP_LAPAZ.csv", index_col=0 )
chrome = pd.read_csv("Chrome.txt",sep="|")
mozilla = pd.read_csv("Firefox.txt", sep="|")
AGENTES = np.concatenate((mozilla.values , chrome.values[2:]))

PCs = []
for i in range(1000): 
    rango_i = RANGOS_LP.sample(1)
    A = rango_i["ip_from"].values[0] + 0 
    B = rango_i["ip_to"].values[0] - 1
    n = np.random.randint(0, B - A ) 
    dir_IP = str(ipaddress.IPv4Address(int(A + n )))
    headers = {  "Host"            : "cnslp.gob.bo:3333"
                ,"User-Agent"      : np.random.choice(AGENTES.flatten(), 1)[0]
                ,"Accept"          : "application/json, text/plain, */*"
                ,"Accept-Language" : "es-MX, es; q = 0.8, en-US; q = 0.5, en; q = 0.3"
                ,"Accept-Encoding" : "gzip, deflate, br"
                ,"Content-Type"    : "application/json"
                ,"Content-Length"  : "29"
                ,"Origin"          : "https://cnslp.gob.bo"
                ,"Connection"      : "keep - alive"
                ,"Referer"         : "https://cnslp.gob.bo/"
        }
    
    PCs.append( { "ip"    :  dir_IP, 
                  "head"  : headers,
                  "cookie": ""})

pc_i = PCs[13]
data_ip = {}
data_ip["ip"] = {"ip" : pc_i["ip"]}
disponible = requests.put(url_disponibilidad+ "/normal",data=json.dumps(data_ip), headers = pc_i["head"],  verify=False )

disponible.json()

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{'ok': True,
 'mensaje': 'Se Recuperaron Registros Correctamente',
 'respuesta': [{'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'},
  {'dia': '04/08/2021'}],
 'cookie': 'Lu23FRxoS4EfKJH8ZLqyhbXF3cM4mL'}

In [12]:
def ObtenerFichaREC(cli_i, Lista_PCs = PCs, ntry = 20):
    """
    Se realiza todo el flujo de consulta para la obtención de fichas 
    """
    ntry -= 1 
    
    if ntry > 0 : 
        try :
            pc_i = get_cookies(np.random.choice(Lista_PCs,1)[0])
            time.sleep(np.random.randint(0,5))
            captcha_i = ResolveCaptcha()
            cli_i["ip"] = {"ip": pc_i[0]["ip"] }
            cli_i["cookie"] = pc_i[1]["cookie"]
            cli_i["recaptcha"] = captcha_i
            respuesta_i = requests.post(url_fichas, cli_i,  verify=False )
        except Exception as exc: 
            print("Reinter por fallo en el request", exc)
            time.sleep(np.random.randint(1,5))

        if respuesta_i.ok :
            salida = respuesta_i.json()["respuesta"]

            oficina_i = oficinasDF.loc[ oficinasDF.id == salida["oficina"], "nombre"].values[0]

            url_i = "http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha={nro_ficha}&fecha_atencion={fecha_atencion}&hora_aproximada={hora_aproximada}&cubiculo={cubiculo}&persona={persona}&oficina={oficina}&codigo={codigo}".format(
                nro_ficha     = salida["numero_atencion"]
                ,fecha_atencion   = pd.to_datetime(salida["fecha"]).strftime("%d/%m/%Y")
                ,hora_aproximada    = salida["hora_aproximada"]
                ,cubiculo  = salida["cubiculo"]
                ,persona  = salida["persona"]
                ,oficina = oficina_i 
                ,codigo = salida["codigo"])
            print(url_i)
        #    URL.append(url_i)
            print(salida["persona"],salida["hora_aproximada"], "\n" )
        else : 
            print("Reintentos restantes {}".format(ntry), cli_i)
            cli_i , respuesta_i = ObtenerFichaREC(cli_i , ntry = ntry)
            
        return cli_i, respuesta_i 
    else : 
        print("Se agotaron la cantidad de reintentos")
        return cli_i, respuesta_i

# Preparando los diccionarios de datos 

In [13]:
fecha_i = '20210804'

fecha_dt = pd.to_datetime(fecha_i, format= "%Y%m%d")

fecha_dt.strftime("%d/%m/%Y")

DF_PENDIENTES = pd.read_csv("{}.csv".format(fecha_i))
DF_PENDIENTES["cubiculo"] = 0
DF_PENDIENTES["atencion" ] = fecha_dt.strftime("%d/%m/%Y")
DF_PENDIENTES["oficina"] = "1"
#DF_PENDIENTES["sexo"] = ""
#DF_PENDIENTES["estado_civil"] = ""
DF_PENDIENTES["direccion"] = ""
DF_PENDIENTES["email"] = ""
DF_PENDIENTES["tipo_atencion"] = "normal"
# DF_PENDIENTES["extension"] = "" # Añadir el condicional de verificar si el campo está vacío 
DF_PENDIENTES["telefono"] = ""
DF_PENDIENTES = DF_PENDIENTES.fillna("")
DF_PENDIENTES.drop("contacto",axis=1,inplace=True)
#DF_PENDIENTES.drop("extencion",axis=1,inplace=True)
DF_PENDIENTES.shape

(3, 17)

In [14]:
DF_PENDIENTES.head()

,ci,extension,emision,ap_paterno,ap_materno,nombre,fecha_nac,celular,estado_civil,sexo,cubiculo,atencion,oficina,direccion,email,tipo_atencion,telefono
0,6140448,,La Paz,RIVEROS,MONTAÑO,CARLOS,1999-03-02,76525031,Soltero,Masculino,0,04/08/2021,1,,,normal,
1,4751201,,La Paz,TAMBO,QUISPE,LIBERATO,1973-12-22,69790016,Casado,Masculino,0,04/08/2021,1,,,normal,
2,9874910,,La Paz,MEDRANO,TICONIPA,MIREYA,1993-10-23,73725896,Casado,Femenino,0,04/08/2021,1,,,normal,


# Ejecución en paralelo

In [15]:
DATOS = [data_i for data_i in DF_PENDIENTES.to_dict(orient="records") ]

In [16]:
TOTAL_RESPUESTAS = []
URL = []

Si todo funciona correctamente hasta aquí, procedemos con la consulta masiva
# CONSULTA PARALELA MASIVA

In [17]:
start = time.time()
processes = []
with ThreadPoolExecutor(max_workers=25) as executor:
    RESULTADOS_FINALES = {executor.submit(ObtenerFichaREC, cli_i) : cli_i for cli_i in DATOS}
    
    for ficha_i in as_completed(RESULTADOS_FINALES):
            url = RESULTADOS_FINALES[ficha_i]
            try:
                data = ficha_i.result()
            except Exception as exc:
                print('{} EXCEPTION {}'.format(url["ci"], exc))
            else:
                print('{} RESULTADO {} '.format(url["ci"], data))    

print(f'Total Ejecución FICHAS : {time.time() - start}')

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

Time taken captcha_i: 54.47155690193176


/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=04/08/2021&hora_aproximada=08:36&cubiculo=Modulo 7&persona=MIREYA MEDRANO TICONIPA&oficina=Regional La Paz&codigo=fZyzl1
MIREYA MEDRANO TICONIPA 08:36 

9874910 RESULTADO ({'ci': 9874910, 'extension': '', 'emision': 'La Paz', 'ap_paterno': 'MEDRANO', 'ap_materno': 'TICONIPA', 'nombre': 'MIREYA', 'fecha_nac': '1993-10-23', 'celular': 73725896, 'estado_civil': 'Casado', 'sexo': 'Femenino ', 'cubiculo': 0, 'atencion': '04/08/2021', 'oficina': '1', 'direccion': '', 'email': '', 'tipo_atencion': 'normal', 'telefono': '', 'ip': {'ip': '190.129.216.54'}, 'cookie': 'o7UjoQuBEHWarhVLqJxC1vEMgl66WQ', 'recaptcha': '03AGdBq27fjfG3Ni-rCMXZGleel6pBYCERlpsPCgHhXruYlPAxTdKT8wDugMOdD2L9cZgZyZFVVUblMeH0nlBScrUBALOD2hAV1KmcmWV7hj1YOU5D7DMkieojpA261e407cmESFMFI5kPO01BP1S9Vqex7KcAzL4hw280BEeDfJ2GVRV6xqiV6zwGAwxNV3WHBO4ycBsWMm0dsvvNcI0f6FNMNEf5vBkbfpW1-SzVUDox3SVQAd0nNTBphJ3XVxiOiPumLEdjBXI9w7WlNVu_44vAQfnR1SJkAyJxZmBRv_JONTgSD3neGiP7qZ

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=04/08/2021&hora_aproximada=08:36&cubiculo=Modulo 8&persona=CARLOS ANDRES RIVEROS MONTAÑO&oficina=Regional La Paz&codigo=OF9NOw
CARLOS ANDRES RIVEROS MONTAÑO 08:36 

6140448 RESULTADO ({'ci': 6140448, 'extension': '', 'emision': 'La Paz', 'ap_paterno': 'RIVEROS', 'ap_materno': 'MONTAÑO', 'nombre': 'CARLOS', 'fecha_nac': '1999-03-02', 'celular': 76525031, 'estado_civil': 'Soltero', 'sexo': 'Masculino', 'cubiculo': 0, 'atencion': '04/08/2021', 'oficina': '1', 'direccion': '', 'email': '', 'tipo_atencion': 'normal', 'telefono': '', 'ip': {'ip': '161.22.188.160'}, 'cookie': 'e2rEYvgAt1lwxbXE8HGpGsL48lI3c1', 'recaptcha': '03AGdBq26KVUjdIO4azt3ziUz5zNk3TdlppNiyxa7wyONTNMMUVTrOhbjyNovdf5p-ZWtHJg07LwJT-KdhSgvVqEtszc2e-oAUBcJnBmEsWzo3q5qixnUYI8AnrLWdqHuDYII0PL8NBgVOw-5ulbU4wIpYeXvqDHxZNXvDpGIiFBfEYEJpdqSzGOChz9mCWTkWR4O9waxNVSMMqAy83aVNNbtgwLvAnYPFFyEVqmkEfLFnInWkpBJc-Nt6G9Oh1QivI0pP-fep7DqCBeiHPlt0hEKaMn_gWIL5ziYIcdCDW5RbR4

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=4&fecha_atencion=04/08/2021&hora_aproximada=08:48&cubiculo=Modulo 1&persona=LIBERATO TAMBO QUISPE&oficina=Regional La Paz&codigo=EoDjqu
LIBERATO TAMBO QUISPE 08:48 

4751201 RESULTADO ({'ci': 4751201, 'extension': '', 'emision': 'La Paz', 'ap_paterno': 'TAMBO', 'ap_materno': 'QUISPE', 'nombre': 'LIBERATO', 'fecha_nac': '1973-12-22', 'celular': 69790016, 'estado_civil': 'Casado', 'sexo': 'Masculino', 'cubiculo': 0, 'atencion': '04/08/2021', 'oficina': '1', 'direccion': '', 'email': '', 'tipo_atencion': 'normal', 'telefono': '', 'ip': {'ip': '190.129.18.238'}, 'cookie': 'j3auONryuWgVpaMznJfAGT9uhaMVBp', 'recaptcha': '03AGdBq27ZEIsSC7JRzrXfD6R2XNtNI4r5sS6G41aqo-7mqQhDhTtON4vgWVQnNJnIZcDAmGEq1GL3iJFeR6KAYhhWrJwoNasuuf_zELd1G9D16tNJuZN0zDqdih2x4mYKahdKR9-8l46qr6LVJ00CTzPFu4_C6lWgxkFsFuLYIsrS-7vnmGmtxHfvCV-jH2J1Ojf6b-YWqu5DSwqqFunvntunVe5vCoQNyvpzb9uZ6ApznC90NzHkiI4Pv72DNnh06_yoNmBgLO4pnVQNZ4LKnUrAXGldE1_wrBmDzAK5SiPd1mfR4IhxPQbkta4KVA-o

In [18]:
ronda_i = [ A for A in as_completed(RESULTADOS_FINALES)]

In [19]:
ronda_i

[<Future at 0x7f2fc15fb2e8 state=finished returned tuple>,
 <Future at 0x7f2fc175b7b8 state=finished returned tuple>,
 <Future at 0x7f2fc15fbba8 state=finished returned tuple>]

In [20]:
[i.result()[1] for i in ronda_i]

[<Response [200]>, <Response [200]>, <Response [200]>]

In [21]:
URLS = []

In [22]:
for i in ronda_i:
    if i.result()[1].ok:
        salida = i.result()[1].json()["respuesta"]

        oficina_i = oficinasDF.loc[ oficinasDF.id == salida["oficina"], "nombre"].values[0]

        url_i = "http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha={nro_ficha}&fecha_atencion={fecha_atencion}&hora_aproximada={hora_aproximada}&cubiculo={cubiculo}&persona={persona}&oficina={oficina}&codigo={codigo}".format(
            nro_ficha     = salida["numero_atencion"]
            ,fecha_atencion   = pd.to_datetime(salida["fecha"]).strftime("%d/%m/%Y")
            ,hora_aproximada    = salida["hora_aproximada"]
            ,cubiculo  = salida["cubiculo"]
            ,persona  = salida["persona"]
            ,oficina = oficina_i 
            ,codigo = salida["codigo"])
        print(url_i)
        URLS.append(url_i)

http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=04/08/2021&hora_aproximada=08:36&cubiculo=Modulo 8&persona=CARLOS ANDRES RIVEROS MONTAÑO&oficina=Regional La Paz&codigo=OF9NOw
http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=04/08/2021&hora_aproximada=08:36&cubiculo=Modulo 7&persona=MIREYA MEDRANO TICONIPA&oficina=Regional La Paz&codigo=fZyzl1
http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=4&fecha_atencion=04/08/2021&hora_aproximada=08:48&cubiculo=Modulo 1&persona=LIBERATO TAMBO QUISPE&oficina=Regional La Paz&codigo=EoDjqu


In [23]:
URLS.sort()
len(URLS)

3

# Obtener PDF basado en el QR 

In [24]:
import os, shutil

In [25]:

for filename in os.listdir(download_directory):
    file_path = os.path.join(download_directory, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [26]:
for i in URLS:
    navegador.get(i)
    time.sleep(1)

In [27]:
len(URLS)

3

In [28]:
# Renombramiento automático

In [29]:
requests.get("http://127.0.0.1:8000/fichasCNS/rename")

<Response [200]>

In [30]:
try: 
    os.rename(  os.path.join(download_directory, "TOTAL.pdf") , 
                os.path.join(download_directory, "TOTAL_{fec}.pdf".format(fec = fecha_i)) )
except : 
    print("Algo malio sal ...")

In [31]:
navegador.close()

# GENERACION DEL ARCHIVO DE LINKS 

In [32]:
fecha_i

'20210804'

In [33]:
DF_PENDIENTES = pd.read_csv("{}.csv".format(fecha_i))

# B = open("E:\\sevima\\SEVIMAworkflow-master\\fichasCNS\\FICHAS\\"+  "LinksMensajes_{}.html".format(fecha_i),"w") # Windows 
B = open("/media/F/sevima/SEVIMAworkflow-master/fichasCNS/FICHAS/"+  "LinksMensajes_{}.html".format(fecha_i),"w")
for data_i in DF_PENDIENTES.to_dict(orient="records") :
    # print("https://wa.me/591{}?text=Buen+dia%2C+adjunta+su+ficha.+Le+recordamos+estar+15+minutos+antes+de+su+hora+de+atenci%C3%B3n+para+no+perder+su+turno.".format(data_i["celular"]))
#    B.writelines('<h1> <a href="https://wa.me/591{}?text=Muy%20buenas%20noches%2C%20lamento%20comunicarle%20que%20el%20sistema%20de%20la%20Caja%20Nacional%20de%20Salud%20parece%20estar%20en%20mantenimiento%20por%20lo%20tanto%20no%20se%20pueden%20sacar%20fichas.%20Esta%20eventualidad%20escapa%20de%20mis%20manos%20para%20poder%20brindarle%20el%20servicio.%20Espero%20que%20ma%C3%B1ana%20se%20habilite%20otra%20vez%20el%20sistema%20y%20en%20funci%C3%B3n%20a%20eso%20yo%20le%20comunico%20las%20novedades%2C%20otra%20vez%20mil%20disculpas.%20"  target="_blank" > {} {} {} {} </a> </h1> <br>'.format(data_i["celular"],data_i["celular"], data_i["nombre"], data_i["ap_paterno"], data_i["contacto"] ))
    B.writelines('<h1> <a href="https://wa.me/591{}?text=Buenas+tardes%2C+adjunta+su+ficha.+Le+recordamos+estar+15+minutos+antes+de+su+hora+de+atenci%C3%B3n+para+no+perder+su+turno."  target="_blank" > {} {} {} {} </a> </h1> <br>'.format(data_i["celular"],data_i["celular"], data_i["nombre"], data_i["ap_paterno"], data_i["contacto"] ))
print("Se genero el archivo exitosamente! ")
B.close()

Se genero el archivo exitosamente! 


In [34]:
# Generando Archivo de compresión
os.system("cd {down_dir} && rar a FICHAS_{fec}.rar *".format(fec = fecha_i, down_dir = download_directory))

0

# Guardando las respuetas 

In [35]:
import pickle
pickle_file = open('respuestas{}.pickle'.format(fecha_i), 'wb')
pickle.dump([i.result() for i in ronda_i], pickle_file)
pickle_file.close()